LangChain 프롬프트 템플릿
```
LLM 프롬프트를 동적으로 구성하고 재사용 할 수 있도록 해주는 도구
    단일입력 : 하나의 변수로 구성된 프롬프트 템플릿
    다중입력 : 둘 이상의 변수를 사용하는 템플릿
    ChatPromptTemplate 역할 기반 프롬프트 : 시스템 / 사용자 역할별 프롬프트 구성  .from_message()
    PartialPeomptTemplate 활용 : 프롬프트 일부를 고정하고 부분 포멧팅을 사용 (ex. 시스템 메세지는 고정..)
    프롬프트 출력 및 체인 실행 : LCEL기법
    프롬프트 작성 팁 : 주의사항 및 모범사례 

```

In [1]:
# 환경설정
%pip install langchain langchain-openai python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# 단일 프롬프트(그냥 from_template) 사용
from langchain.prompts import PromptTemplate

# 템플릿 문자열 정의
template_str = (
    "당신은 최고 수준의 마케팅 카피아리터 입니다.\n"
    "아래 제품에 대한 매력적인 홍보문구를 100자 내외로 작성해 주세요. \n\n"
    "제품명 : {product_name}"
)

# 프롬프트 템플릿 객체 생성
product_prompt = PromptTemplate.from_template(template_str)

# 템플릿에 값 채우기 
formatted_prompt = product_prompt.format(product_name='이클립스')
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피아리터 입니다.
아래 제품에 대한 매력적인 홍보문구를 100자 내외로 작성해 주세요. 

제품명 : 이클립스


In [10]:
# llm에 연결하기
# LCEL
# 프롬프트 | llm  => invoke 사용


from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

# Runnable객체 생성 LCEL
chain = product_prompt |llm
result = chain.invoke({'product_name': '이클립스'})
print(result.content)


"이클립스, 당신의 일상에 빛나는 변화를! 매일의 순간을 특별하게 만들어주는 완벽한 선택. 지금 경험해보세요!"


In [15]:
# 다중 입력(단일은 from_template 사용했지만 다중은 생성자를 이용하는게 좋음)
# 다중입력 템플릿 문자열 정의
from langchain.prompts import ChatPromptTemplate

multi_template_str = (
    '아래는 뉴스 기사 제목과 키워드 입니다. \n'
    '이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요 \n\n'
    '제목 : {title}'
    '키워드 : {keyword}'
)

# 프롬프트 템플릿 
summary_prompt = PromptTemplate(template=multi_template_str, input_variables=['title','keyword'])

# 포멧팅을 통해 프롬프트 값 확인
sample_title = '인공지능 기술의 발전과 미래'
sample_keyword = '머신러닝, 딥러닝, LLM, 랭체인, 산업 혁신'
formatted_summary_prompt = summary_prompt.format(title = sample_title, keyword = sample_keyword)
print(formatted_summary_prompt)


아래는 뉴스 기사 제목과 키워드 입니다. 
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요 

제목 : 인공지능 기술의 발전과 미래키워드 : 머신러닝, 딥러닝, LLM, 랭체인, 산업 혁신


In [21]:
# LCEL 출력
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
result_summary = (summary_prompt | llm ).invoke({
    'title': sample_title,
    'keyword': sample_keyword
})
result_summary

AIMessage(content='인공지능 기술의 발전은 머신러닝과 딥러닝의 혁신을 통해 가속화되고 있으며, 특히 대규모 언어 모델(LLM)과 랭체인 기술이 주목받고 있습니다. 이러한 기술들은 다양한 산업 분야에서 혁신을 이끌어내고 있으며, 기업들은 이를 활용하여 효율성을 높이고 새로운 비즈니스 모델을 창출하고 있습니다. 앞으로 인공지능은 더욱 발전하여 우리의 삶과 산업 구조에 큰 변화를 가져올 것으로 기대됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 76, 'total_tokens': 187, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BbKhBxxdZPzS5tGBFqjrOJGaVlE74', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--30b516a7-304f-445b-8c33-3901a3002864-0', usage_metadata={'input_tokens': 76, 'output_tokens': 111, 'total_tokens': 187, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 

In [17]:
# ChatPromptTemplate과 역할 기반 프롬프트
# 시스템/사용자/어시스턴트 역할 (role)을 포함한 다중 템플릿을 구성하는 프롬프트
# 시스템 메시지: 모델의 동작을 지시
# 사용자 메시지: 실제 사용자의 입력
# 어시스턴트 메시지: 이전 모델이 응답한 내용이 있다면 대화 맥락 유지를 위해 활용 



In [23]:
system_message = ('당신은 python 분야의 뛰어난 전문가이자 조언자 입니다.'
                  '사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요')
user_message = '{question}'
chat_prompt = ChatPromptTemplate.from_messages([
    ('system', system_message),
    ('user', user_message)
])

# 템플릿을 이용해서 문장을 완성해 보세요 => 사용법을 알기위해서 하는것 llm에서는 사용 안함
sample_question = '파이선의 클래스에 대해서 설명해주세요'
formated_message = chat_prompt.format_messages(question=sample_question)
print(formated_message)

# 파이프라인을 이용해서 llm 호출 및 출력
answer = (chat_prompt | llm | parser).invoke({'question': sample_question})
print(answer)

[SystemMessage(content='당신은 python 분야의 뛰어난 전문가이자 조언자 입니다.사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 답변해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이선의 클래스에 대해서 설명해주세요', additional_kwargs={}, response_metadata={})]
파이썬의 클래스는 객체 지향 프로그래밍(OOP)의 기본 개념 중 하나로, 객체를 생성하기 위한 청print(blueprint)입니다. 클래스는 속성(데이터)과 메서드(함수)를 포함할 수 있으며, 이를 통해 관련된 데이터와 기능을 하나의 단위로 묶을 수 있습니다.

### 클래스의 기본 구조

클래스를 정의하려면 `class` 키워드를 사용합니다. 다음은 간단한 클래스의 예입니다:

```python
class Dog:
    # 생성자 메서드
    def __init__(self, name, age):
        self.name = name  # 인스턴스 변수
        self.age = age    # 인스턴스 변수

    # 메서드
    def bark(self):
        return f"{self.name} says woof!"

# 클래스의 인스턴스 생성
my_dog = Dog("Buddy", 3)

# 인스턴스 메서드 호출
print(my_dog.bark())  # 출력: Buddy says woof!
```

### 주요 개념

1. **생성자 (`__init__` 메서드)**:
   - 클래스의 인스턴스가 생성될 때 호출되는 특별한 메서드입니다.
   - 인스턴스 변수(속성)를 초기화하는 데 사용됩니다.

2. **인스턴스 변수**:
   - 각 인스턴스(객체)마다 고유한 데이터를 저장하는 변수입니다.
   - `self` 키워드를 사용하여 인스턴스 변수를 정의합니다.

3. **메서드**:
   - 클래스 내에서 

In [24]:
# ParitalPtomptTemplate : 템플릿의 일부를 부분적으로 채운 새로운 탬플릿
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
role_system_template = '당신은 {role} 분야의 전문 지식인 입니다. 가능한 자세히 답변해 주세요'
system_prompt = SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template('{question}')

# chatprompttemplate을 생성
base_chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
partial_chat_prompt =  base_chat_prompt.partial(role = '주식투자')

In [25]:
#partial로 생성된 프롬프트에 질문만 채워 프롬프트 구성
sample_question = "현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은"
message = partial_chat_prompt.format_messages(question=sample_question)
print(message)

[SystemMessage(content='당신은 주식투자 분야의 전문 지식인 입니다. 가능한 자세히 답변해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은', additional_kwargs={}, response_metadata={})]


In [26]:
# LCEL - invoke
answer = (partial_chat_prompt | llm | parser).invoke({'question':sample_question})
print(answer)


2025년 5월의 삼성전자 주식 전망을 평가하기 위해서는 여러 가지 요소를 고려해야 합니다. 다음은 삼성전자의 주식 전망에 영향을 미칠 수 있는 주요 요인들입니다.

1. **반도체 시장 동향**: 삼성전자는 세계 최대의 반도체 제조업체 중 하나입니다. 반도체 수요는 AI, IoT, 5G 등 다양한 기술 발전에 따라 증가하고 있습니다. 2025년에는 이러한 기술들이 더욱 발전할 것으로 예상되며, 이에 따라 반도체 수요가 증가할 가능성이 높습니다. 그러나 반도체 시장은 경기 사이클에 민감하므로, 글로벌 경제 상황에 따라 변동성이 클 수 있습니다.

2. **스마트폰 및 가전제품 판매**: 삼성전자는 스마트폰과 가전제품에서도 강력한 입지를 가지고 있습니다. 2025년에는 새로운 모델과 혁신적인 기술이 출시될 가능성이 있으며, 이는 매출 증가에 긍정적인 영향을 미칠 수 있습니다. 특히, 폴더블 스마트폰과 같은 새로운 형태의 제품이 소비자에게 인기를 끌 수 있습니다.

3. **글로벌 경제 상황**: 글로벌 경제의 성장률, 금리, 인플레이션 등은 삼성전자의 주가에 큰 영향을 미칠 수 있습니다. 경제가 안정적이고 성장세를 보인다면 소비자 지출이 증가하고, 이는 삼성전자의 매출 증가로 이어질 수 있습니다.

4. **경쟁 상황**: 삼성전자는 애플, 화웨이, TSMC 등과 같은 강력한 경쟁자들과 경쟁하고 있습니다. 이들 기업의 기술 발전과 시장 점유율 변화는 삼성전자의 주가에 영향을 미칠 수 있습니다.

5. **정치적 및 규제 환경**: 한국과 글로벌 시장에서의 정치적 안정성, 무역 정책, 규제 변화 등도 삼성전자의 운영에 영향을 미칠 수 있습니다. 특히, 반도체 산업은 국가 간의 무역 갈등에 민감하므로, 이러한 요소를 주의 깊게 살펴봐야 합니다.

6. **ESG(환경, 사회, 지배구조) 경영**: 최근 투자자들은 ESG 요소를 중요하게 고려하고 있습니다. 삼성전자가 지속 가능한 경영을 통해 긍정적인 이미지를 구축하고, 이에 따라 투자자들의 신뢰를 얻는다면 주가에 긍

```
이전 대화 내용을 기억해서 문맥을 유지하는 역할
LangChain 0.3x부터는 LCEL 기반으로 체인을 구성
RunnableWithMessageHistory , ChatMessageHistory 등의 컴포넌트를 활용해서 세션별 대화 기록을 관리
대화가 장기화될 경우 요약 메모리를 도입해서 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이문제를 해결
```

```
%pip install langchain langchain-openai python-dotenv langchain_redis --quiet 로 인스톨하고 
env환경 가져와야하니까 from dotenv import load_dotenv 해줌
랭체인 코어에 점 하면 챗 히스토리가 있음.
from langchain_core.chat_history import InMemoryChatMessageHistory
```

In [77]:
%pip install langchain langchain-openai python-dotenv langchain_redis --quiet

Note: you may need to restart the kernel to use updated packages.


In [78]:
from dotenv import load_dotenv
load_dotenv()

True

코드에 있는 msg.type과 msg.content는 Langchain의 ChatMessage 객체가 갖고 있는 속성(attributes)

🔍 msg.type
누가 보낸 메시지인지를 나타내는 문자열
주로 두 가지 값 중 하나가 나와:
"human" → 사용자가 보낸 메시지
"ai" → AI가 보낸 메시지
👉 즉, 사람/AI 구분용이야.

🔍 msg.content
실제 메시지 본문 내용, 즉 텍스트 문자열이야.

In [79]:

from langchain_core.chat_history import  InMemoryChatMessageHistory
# 메모리 객체 생성
history =  InMemoryChatMessageHistory()
history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
history.add_ai_message('안녕하세요 홍길동님, 무엇을 도와드릴까요?')
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

human : 안녕하세요 제 이름은 홍길동 입니다.
ai : 안녕하세요 홍길동님, 무엇을 도와드릴까요?


In [81]:
# Radis 기반 채팅 기록 저장소
from langchain_redis import RedisChatMessageHistory
import os
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
session_id = 'user_123'
history =  RedisChatMessageHistory(session_id=session_id, redis_url=REDIS_URL)
history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
history.add_ai_message('안녕하세요 홍길동님, 무엇을 도와드릴까요?')
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

ConnectionError: Error 10061 connecting to localhost:6379. 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다.

In [87]:
# 세션기반 다중사용자 메모리 구조 구현 - 다중사용자 챗봇
# 핵심 :session_id를 키로 하는 메모리 저장소만들고 사용자의 대화는 키별로 저장한다.
from langchain_core.prompts import  ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system",'당신은 뛰어난 한국어 상담 챗봇입니다 질문에 친절하고 자세히 답변해주세요'),
    # history키로 전달된 메세지 목록은 체인 실행시 해당 위치에 넣겠다는 의미
    MessagesPlaceholder(variable_name='history'), 
    ('human','{input}')
])
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0)


In [88]:
# LCEL
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [89]:
# 세션별 메모리 저장소를 딕셔너리로 만들고, 존재하지 않는 새로운 세션 id가 들어오면 InMemoryChatMessageHistory를 생성
# get_session_history를 구현


In [99]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# 세션 id -> 대화 기록 객체 매핑
store = {}
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    '''"세션 ID"에 해당하는 대화 기록 객체를 반환합니다.(없으면 새로 생성)'''
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메모리를 통합한 체인 래퍼 생성
chatbot = RunnableWithMessageHistory(    
    chain,
    get_session_history,    
    input_messages_key = 'input',
    history_messages_key = 'history'
)

In [100]:

# 두개의 세션을 번갈아가면서 대화  RunnableWithMessageHistory 가 각 세션에 맞는 대화 기록을 관리합니다.
sessions = ['user_a','user_b']
questions = [
    '안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?',   # usre_a 첫번재 질문
    '안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?', # user_b 첫번째 질문
    '저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?', # user_a 두번째 질문
    '저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?' # user_b 두번째 질문
]
for i, question in enumerate(questions):
    session_id = sessions[i % 2]  # 세션 ID를 번갈아가며 사용
    result = chatbot.invoke({'input': question}, config={'configurable': {'session_id': session_id}})
    print(f'[{session_id}] 질문: {question}')
    print(f'[{session_id}] 챗봇: {result}\n')

17:48:15 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_a] 질문: 안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?
[user_a] 챗봇: 안녕하세요, 홍길동님! 저는 여러분의 질문에 답변하고 도움을 드리기 위해 만들어진 챗봇입니다. 어떤 궁금한 점이나 도움이 필요하신 부분이 있으신가요?

17:48:17 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_b] 질문: 안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?
[user_b] 챗봇: 안녕하세요, 이순신님! 저는 여러분의 질문에 답변하고, 정보 제공, 상담 등을 도와주는 챗봇입니다. 궁금한 점이나 도움이 필요한 부분이 있다면 언제든지 말씀해 주세요!

17:48:19 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_a] 질문: 저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?
[user_a] 챗봇: 프로그래밍을 배우고 계시다니 멋지네요! 저는 여러분의 질문에 답변하고, 정보 제공, 문제 해결, 그리고 다양한 주제에 대한 상담을 하는 역할을 하고 있습니다. 프로그래밍에 관련된 질문이나 도움이 필요하시면 언제든지 말씀해 주세요! 어떤 언어를 배우고 계신가요?

17:48:20 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_b] 질문: 저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?
[user_b] 챗봇: 역사에 관심이 많으시다니 정말 멋지네요! 저는

In [101]:
result = chatbot.invoke({'input':"저는 철수예요, 반갑습니다"}, config={'configurable' : {'session_id' : 'user_c'}})
print(f'[user_c] 질문: 저는 철수예요, 반갑습니다.')
print(f'[user_c] 챗봇: {result}\n')

17:48:22 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_c] 질문: 저는 철수예요, 반갑습니다.
[user_c] 챗봇: 안녕하세요, 철수님! 반갑습니다. 어떻게 도와드릴까요? 궁금한 점이나 이야기하고 싶은 주제가 있다면 말씀해 주세요.



In [102]:
result = chatbot.invoke({'input':"저는 누구라고요?"}, config={'configurable' : {'session_id' : 'user_a'}})
# user_a인 홍길동과의 과거 기록을 토대로 ' 당신은 홍길동입니다'라고 대답해야함
result

17:48:23 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'홍길동님이라고 말씀하셨습니다! 혹시 더 궁금한 점이나 다른 질문이 있으신가요? 도움이 필요하시면 언제든지 말씀해 주세요.'

In [103]:
result = chatbot.invoke({'input':"저는 철수예요, 반갑습니다"}, config={'configurable' : {'session_id' : 'user_c'}})
print(f'[user_c] 질문: 저는 철수예요, 반갑습니다.')
print(f'[user_c] 챗봇: {result}\n')

17:48:25 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_c] 질문: 저는 철수예요, 반갑습니다.
[user_c] 챗봇: 안녕하세요, 철수님! 다시 만나서 반갑습니다. 오늘은 어떤 이야기를 나누고 싶으신가요? 궁금한 점이나 도움이 필요한 부분이 있다면 말씀해 주세요!



요약 메모리 구현(대화내용 자동 요약)
```
긴 대화내용을 모두 프롬프트에 기록하는 것은 비효율적 -> 프롬프트의 길이 제한에 걸릴 가능성이 있음
Conversation Summary Memory
0.3x 버전에서는 직접 요약용 체인을 만들어서 ChatMessageHistory에 적용
```

어떻게 요약?
```
- 일정길이 이상으로 대화가 누적되면, 과거 대화를 요약해서 핵심 내용만 남김
- 요약 결과를 메모리에 시스템 메시지 등으로 저장 -> 메모리 절약
- 새로운 사용자 입력시 요약된 맥락 + 최근 몇 메시지만 참고해서 llm 전달


In [104]:
# 요약용 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 대화 요약 전문가입니다. 대화의 주요 내용을 간결하게 요약해 주세요"),
    ("human","{conversation}") #전체 대화내용을 하나의 문자열로 전달
])
# LCEL
summary_chain = summary_prompt | llm | StrOutputParser()

In [105]:
# user_d 세션에 대화내용을 기록 긴 대화 생성
long_queries = [
    '안녕, 오늘 우리 뭐하려고 했지?',
    '아 맞다 내일 회의자료 준비해야지, 회의는 몇시지?',
    '그 회의에 누가 참석하는지 기억나니?',
    '단위 프로젝트 진행 상황도 공유해야 할까?',
    '최근에 이야기 했던 새로운 기능에 대한 업데이트는 있어?'
]

session_id = 'user_d'
for q in long_queries:
    answer = chatbot.invoke({'input':q}, config={'configurable' : {'session_id': session_id}})


print(f'요약전 user_d의 메모리 메세지 개수: {len(store[session_id].messages)}')
print(store[session_id]) #요약하기전 대화 내용

17:48:27 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:48:28 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:48:30 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:48:33 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:48:36 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
요약전 user_d의 메모리 메세지 개수: 10
Human: 안녕, 오늘 우리 뭐하려고 했지?
AI: 안녕하세요! 오늘 어떤 이야기를 나누고 싶으신가요? 궁금한 점이나 하고 싶은 주제가 있다면 말씀해 주세요. 함께 이야기해보아요!
Human: 아 맞다 내일 회의자료 준비해야지, 회의는 몇시지?
AI: 회의 시간이 기억나지 않으신가요? 회의 일정은 보통 이메일이나 캘린더에 기록되어 있을 텐데요. 확인해보시고, 필요한 자료나 준비물도 함께 점검해보시면 좋을 것 같아요. 혹시 회의 자료 준비에 도움이 필요하시면 말씀해 주세요!
Human: 그 회의에 누가 참석하는지 기억나니?
AI: 회의 참석자에 대한 정보는 제가 알 수 없지만, 보통 회의 초대 이메일이나 캘린더에 참석자 목록이 포함되어 있을 거예요. 그곳에서 확인해보시면 좋을 것 같습니다. 만약 참석자 목록을 정리하거나 회의 자료를 준비하는 데 도움이 필요하시면 말씀해 주세요!
Human: 단위 프로젝트 진행 상황도 

In [107]:
# 전체 대화 내용을 요약하고 마지막 사용자 질문-답변 쌍만 원본 유지
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
# 요약 대상 대화 내용 추출(마지막 QA 쌍 제외한 이전 내용)
message = store[session_id].messages

# 1번 주고받은 대화 이후에 또 했다는 것 -> 마지막을 빼야함
if len(message) > 2 :
    original_dialog = '\n'.join([f'{msg.type.upper()} : {msg.content}' for msg in message[ : -2]])
else : 
    original_dialog = '\n'.join([f'{msg.type.upper()} : {msg.content}' for msg in message])

# llm으로 요약 생성
summary_text = summary_chain.invoke({'conversation' : original_dialog})
print("== 요약 내용 ==")
print(summary_text)
# 기존 메모리를 요약으로 교체 : 이전 내용 요약본 + 최근 QA 유지
new_history = InMemoryChatMessageHistory()
new_history.messages.append(SystemMessage(content = f'요약: {summary_text}'))

# 최근 대화의 마지막 QA쌍 복원
if len(message) >= 2 :
    last_user_msg = message[-2]
    last_ai_msg = message[-1]
    # 휴먼 메시지인지 검사
    if isinstance(last_user_msg, HumanMessage) :
        new_history.add_user_message(last_user_msg.content)
    else :
        new_history.messages.append(last_user_msg)
    if isinstance(last_ai_msg, AIMessage) :
        new_history.add_ai_message(last_ai_msg.content)
    else :
        new_history.messages.append(last_ai_msg)
# 메모리 교체
store[session_id] = new_history

17:50:16 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
== 요약 내용 ==
대화 요약: 
사용자는 내일 회의 자료를 준비해야 한다고 언급하며 회의 시간과 참석자를 확인하려고 한다. AI는 회의 일정과 참석자 목록은 이메일이나 캘린더에서 확인할 수 있다고 안내하고, 단위 프로젝트 진행 상황을 공유하는 것이 좋다고 제안한다. AI는 회의에서 공유할 내용을 정리하는 방법을 제시하며, 필요한 경우 도움을 제공하겠다고 한다.
